# KBG Syndrome

Data from [Martinez-Cayuelas E, et al. Clinical description, molecular delineation and genotype-phenotype correlation in 340 patients with KBG syndrome: addition of 67 new patients](https://pubmed.ncbi.nlm.nih.gov/36446582).

We investigate subjects with mutations in *ANKRD11*.

In [1]:
import genophenocorr

print(f"Using genophenocorr version {genophenocorr.__version__}")

Using genophenocorr version 0.1.1dev


## Setup

### Load HPO

We use HPO `v2023-10-09` release for this analysis.

In [2]:
import hpotk

fpath_hpo = 'https://github.com/obophenotype/human-phenotype-ontology/releases/download/v2023-10-09/hp.json'
hpo = hpotk.load_minimal_ontology(fpath_hpo)

### Load Phenopackets

We will load phenopacket JSON files located in `phenopackets` folder that is next to the notebook.

In [3]:
from genophenocorr.preprocessing import configure_caching_cohort_creator, load_phenopacket_folder

fpath_phenopackets = 'phenopackets'
cohort_creator = configure_caching_cohort_creator(hpo, cache_dir='temp_cache')
cohort = load_phenopacket_folder(fpath_phenopackets, cohort_creator)

Patients Created: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 328/328 [00:01<00:00, 234.42it/s]
Validated under none policy
Showing errors and warnings
328 phenopacket(s) found at `phenopackets`
  patient #0
    variants
     warnings:
     ·Expected a VCF record, a VRS CNV, or an expression with `hgvs.c` but had an error retrieving any from patient Novara, 2017_P2[PMID_36446582_Novara,_2017_P2]. Remove variant from testing
     ·Patient PMID_36446582_Novara,_2017_P2 has no variants to work with
  patient #1
    variants
     warnings:
     ·Expected a VCF record, a VRS CNV, or an expression with `hgvs.c` but had an error retrieving any from patient Goldenberg2016_P13[PMID_36446582_Goldenberg2016_P13]. Remove variant from testing
     ·Patient PMID_36446582_Goldenberg2016_P13 has no variants to work with
  patient #3
    variants
     warnings:
     ·Expected a VCF record, a VRS CNV, or an expression with `hgvs.c` b

### Pick transcript

We choose the [MANE Select](https://www.ncbi.nlm.nih.gov/nuccore/NM_013275.6) transcript for *ANKRD11*.

In [4]:
tx_id = 'NM_013275.6'

## Explore cohort

Explore the cohort to guide selection of the genotype-phenotype analysis.


In [5]:
from IPython.display import HTML, display
from genophenocorr.view import CohortViewer

viewer = CohortViewer(hpo)

In [6]:
display(HTML(viewer.cohort_summary_table(cohort)))

Item,Description
Total Individuals,328
Excluded Individuals,"77: Scarano, 2013_P12[PMID_36446582_Scarano,_2013_P12];Goldenberg2016_P10[PMID_36446582_Goldenberg2016_P10];Khalifa, 2013_P1B[PMID_36446582_Khalifa,_2013_P1B];Bucerzan2020[PMID_36446582_Bucerzan2020];Goldenberg2016_P33[PMID_36446582_Goldenberg2016_P33];Isrie, 2012_P1[PMID_36446582_Isrie,_2012_P1];KBG38[PMID_36446582_KBG38];Gnazzo, 2020_P29[PMID_36446582_Gnazzo,_2020_P29];Kutkowska-Kazmierczak2021_P14[PMID_36446582_Kutkowska-Kazmierczak2021_P14];Novara, 2017_P4[PMID_36446582_Novara,_2017_P4];Sacharow, 2012_P1[PMID_36446582_Sacharow,_2012_P1];Lim2014[PMID_36446582_Lim2014];Goldenberg2016_P36[PMID_36446582_Goldenberg2016_P36];Willemsen2010_P2[PMID_36446582_Willemsen2010_P2];Goldenberg2016_P22[PMID_36446582_Goldenberg2016_P22];Kutkowska-Kazmierczak2021_P17[PMID_36446582_Kutkowska-Kazmierczak2021_P17];Goldenberg2016_P32[PMID_36446582_Goldenberg2016_P32];Kutkowska-Kazmierczak2021_P23[PMID_36446582_Kutkowska-Kazmierczak2021_P23];Ockeloen2015_P20[PMID_36446582_Ockeloen2015_P20];Novara, 2017_P5[PMID_36446582_Novara,_2017_P5];KBG1[PMID_36446582_KBG1];Khalifa, 2013_P1A[PMID_36446582_Khalifa,_2013_P1A];Willemsen2010_P1[PMID_36446582_Willemsen2010_P1];Novara, 2017_P11[PMID_36446582_Novara,_2017_P11];Goldenberg2016_P20[PMID_36446582_Goldenberg2016_P20];Goldenberg2016_P2[PMID_36446582_Goldenberg2016_P2];KBG23[PMID_36446582_KBG23];Kutkowska-Kazmierczak2021_P19[PMID_36446582_Kutkowska-Kazmierczak2021_P19];KBG26[PMID_36446582_KBG26];Novara, 2017_P2[PMID_36446582_Novara,_2017_P2];KBG9[PMID_36446582_KBG9];Isrie, 2012_P2[PMID_36446582_Isrie,_2012_P2];KBG25[PMID_36446582_KBG25];Kutkowska-Kazmierczak2021_P20[PMID_36446582_Kutkowska-Kazmierczak2021_P20];Scarano, 2013_P11[PMID_36446582_Scarano,_2013_P11];Goldenberg2016_P13[PMID_36446582_Goldenberg2016_P13];Goldenberg2016_P26[PMID_36446582_Goldenberg2016_P26];Scarano, 2013_P10[PMID_36446582_Scarano,_2013_P10];Goldenberg2016_P21[PMID_36446582_Goldenberg2016_P21];KBG2[PMID_36446582_KBG2];Crippa2015_P3[PMID_36446582_Crippa2015_P3];Spengler, 2013[PMID_36446582_Spengler,_2013];Goldenberg2016_P29[PMID_36446582_Goldenberg2016_P29];Sacharow, 2012_P2[PMID_36446582_Sacharow,_2012_P2];KBG58[PMID_36446582_KBG58];Parenti2021_P23[PMID_36446582_Parenti2021_P23];Novara, 2017_P7[PMID_36446582_Novara,_2017_P7];Palumbo 2016[PMID_36446582_Palumbo_2016];Goldenberg2016_P19[PMID_36446582_Goldenberg2016_P19];Kutkowska-Kazmierczak2021_P22[PMID_36446582_Kutkowska-Kazmierczak2021_P22];Gnazzo, 2020_P31[PMID_36446582_Gnazzo,_2020_P31];Crippa2015_P1[PMID_36446582_Crippa2015_P1];Behnert, 2018[PMID_36446582_Behnert,_2018];Novara, 2017_P12[PMID_36446582_Novara,_2017_P12];Novara, 2017_P8[PMID_36446582_Novara,_2017_P8];Kutkowska-Kazmierczak2021_P15[PMID_36446582_Kutkowska-Kazmierczak2021_P15];Srivastava, 2017_P1[PMID_36446582_Srivastava,_2017_P1];Goldenberg2016_P12[PMID_36446582_Goldenberg2016_P12];Willemsen2010_P3[PMID_36446582_Willemsen2010_P3];Goldenberg2016_P3[PMID_36446582_Goldenberg2016_P3];Kutkowska-Kazmierczak2021_P16[PMID_36446582_Kutkowska-Kazmierczak2021_P16];Novara, 2017_P9[PMID_36446582_Novara,_2017_P9];Novara, 2017_P1[PMID_36446582_Novara,_2017_P1];Youngs2011[PMID_36446582_Youngs2011];KBG22[PMID_36446582_KBG22];Willemsen2010_P4[PMID_36446582_Willemsen2010_P4];Kutkowska-Kazmierczak2021_P21[PMID_36446582_Kutkowska-Kazmierczak2021_P21];Goldenberg2016_P4[PMID_36446582_Goldenberg2016_P4];Goldenberg2016_P18[PMID_36446582_Goldenberg2016_P18];Goldenberg2016_P1[PMID_36446582_Goldenberg2016_P1];Kutkowska-Kazmierczak2021_P18[PMID_36446582_Kutkowska-Kazmierczak2021_P18];Gnazzo, 2020_P30[PMID_36446582_Gnazzo,_2020_P30];Goldenberg2016_P28[PMID_36446582_Goldenberg2016_P28];Goldenberg2016_P24[PMID_36446582_Goldenberg2016_P24];Miyatake, 2013[PMID_36446582_Miyatake,_2013];Crippa2015_P2[PMID_36446582_Crippa2015_P2];Novara, 2017_P3[PMID_36446582_Novara,_2017_P3]"
Total Unique HPO Terms,27
Total Unique Variants,251


In [7]:
display(HTML(viewer.hpo_term_counts_table(cohort))) ## Add Labels to output

HPO Term,Count
Macrodontia (HP:0001572),182
Intellectual disability (HP:0001249),159
Abnormality of the hand (HP:0001155),156
Global developmental delay (HP:0001263),133
Short stature (HP:0004322),115
Abnormal external nose morphology (HP:0010938),112
Thick eyebrow (HP:0000574),105
Long philtrum (HP:0000343),103
Hearing impairment (HP:0000365),74
Triangular face (HP:0000325),68


In [8]:
display(HTML(viewer.variants_table(cohort, tx_id))) 

Variant,Effect,Count,Key
c.1903_1907del,FRAMESHIFT_VARIANT,33,16_89284634_89284639_GTGTTT_G
c.2408_2412del,FRAMESHIFT_VARIANT,10,16_89284129_89284134_CTTTTT_C
c.2398_2401del,FRAMESHIFT_VARIANT,8,16_89284140_89284144_TTTTC_T
c.1381_1384del,FRAMESHIFT_VARIANT,8,16_89285157_89285161_GTTTC_G
c.6792_6793insC,FRAMESHIFT_VARIANT,5,16_89279749_89279749_C_CG
c.7481_7482insC,FRAMESHIFT_VARIANT,5,16_89275180_89275180_A_AG
c.2182_2183del,FRAMESHIFT_VARIANT,3,16_89284358_89284360_GAT_G
c.3832A>T,STOP_GAINED,3,16_89282710_89282710_T_A
c.3224_3227del,FRAMESHIFT_VARIANT,3,16_89283314_89283318_CCTTT_C
c.2175_2178del,FRAMESHIFT_VARIANT,3,16_89284363_89284367_CTTTG_C


## Configure the analysis

In [10]:
from genophenocorr.analysis import configure_cohort_analysis, CohortAnalysisConfiguration
from genophenocorr.analysis.predicate import BooleanPredicate, GroupingPredicate

analysis_config = CohortAnalysisConfiguration.builder()\
    .missing_implies_excluded(True)\
    .pval_correction('fdr_bh')\
    .min_perc_patients_w_hpo(0.1)\
    .build()
analysis = configure_cohort_analysis(cohort, hpo, config=analysis_config)

Test for presence of genotype-phenotype correlations between frameshift variants vs. others.

In [11]:
from genophenocorr.model import VariantEffect

frameshift = analysis.compare_by_variant_effect(VariantEffect.FRAMESHIFT_VARIANT, tx_id)
frameshift.summarize(hpo, BooleanPredicate.YES)

FRAMESHIFT_VARIANT on NM_013275.6                    No                 Yes  \
                                                  Count    Percent    Count   
Intellectual disability [HP:0001249]             60/238  25.210084   99/238   
Abnormality of mental function [HP:0011446]      66/240  27.500000  113/240   
Neurodevelopmental abnormality [HP:0012759]      70/242  28.925620  127/242   
Abnormal nervous system physiology [HP:0012638]  73/244  29.918033  135/244   
Abnormality of the nervous system [HP:0000707]   73/244  29.918033  135/244   
Global developmental delay [HP:0001263]          47/204  23.039216   86/204   
Neurodevelopmental delay [HP:0012758]            47/204  23.039216   86/204   
Abnormal external nose morphology [HP:0010938]   42/212  19.811321   70/212   
Abnormal nasal morphology [HP:0005105]           42/212  19.811321   70/212   
Abnormality of the nose [HP:0000366]             42/212  19.811321   70/212   
Abnormality of the hand [HP:0001155]             56/241  23.236515  100/241   
Abnormality of the upper limb [HP:0002817]       56/241  23.236515  100/241   
Abnormality of limbs [HP:0040064]                56/241  23.236515  100/241   
Long philtrum [HP:0000343]                       37/216  17.129630   66/216   
Abnormality of the philtrum [HP:0000288]         37/216  17.129630   66/216   
Abnormal upper lip morphology [HP:0000177]       37/216  17.129630   66/216   
Abnormal lip morphology [HP:0000159]             37/216  17.129630   66/216   
Abnormal ear morphology [HP:0031703]             21/196  10.714286   37/196   
Abnormal hair morphology [HP:0001595]            36/210  17.142857   69/210   
Abnormal ocular adnexa morphology [HP:0030669]   36/210  17.142857   69/210   
Abnormal skin adnexa morphology [HP:0011138]     36/210  17.142857   69/210   
Abnormality of the ocular adnexa [HP:0032039]    36/210  17.142857   69/210   
Abnormality of the integument [HP:0001574]       36/210  17.142857   69/210   
Abnormality of the orbital region [HP:0000315]   36/210  17.142857   69/210   
Atypical behavior [HP:0000708]                   24/190  12.631579   48/190   
Triangular face [HP:0000325]                     23/194  11.855670   45/194   
Abnormal facial shape [HP:0001999]               23/194  11.855670   45/194   
Short stature [HP:0004322]                       40/229  17.467249   75/229   
Abnormality of body height [HP:0000002]          40/229  17.467249   75/229   
Growth delay [HP:0001510]                        40/229  17.467249   75/229   
Growth abnormality [HP:0001507]                  40/229  17.467249   75/229   
Cognitive impairment [HP:0100543]                16/187   8.556150   35/187   
Phenotypic abnormality [HP:0000118]              80/251  31.872510  168/251   
All [HP:0000001]                                 80/251  31.872510  168/251   
Abnormality of the ear [HP:0000598]              34/209  16.267943   77/209   
Abnormality of the face [HP:0000271]             70/242  28.925620  150/242   
Abnormality of the head [HP:0000234]             70/242  28.925620  150/242   
Abnormality of head or neck [HP:0000152]         70/242  28.925620  150/242   
Abnormal oral cavity morphology [HP:0000163]     65/240  27.083333  140/240   
Abnormal oral morphology [HP:0031816]            65/240  27.083333  140/240   
Abnormality of the mouth [HP:0000153]            65/240  27.083333  140/240   
Hearing abnormality [HP:0000364]                 22/198  11.111111   52/198   
Abnormal ear physiology [HP:0031704]             22/198  11.111111   52/198   
Macrodontia [HP:0001572]                         58/236  24.576271  124/236   
Abnormal dental morphology [HP:0006482]          58/236  24.576271  124/236   
Abnormality of the dentition [HP:0000164]        58/236  24.576271  124/236   

FRAMESHIFT_VARIANT on NM_013275.6                                     \
                                                   Percent   p value   
Intellectual disability [HP:0001249]             41.596639  0.0

Test for presence of genotype-phenotype correlations between subjects with >=1 allele of a variant vs. the other subjects:


In [12]:
var_single = analysis.compare_by_variant_key('16_89284634_89284639_GTGTTT_G')
var_single.summarize(hpo, BooleanPredicate.YES)

>=1 allele of the variant 16_89284634_89284639_GTGTTT_G       No             \
                                                           Count    Percent   
Neurodevelopmental abnormality [HP:0012759]              176/242  72.727273   
Abnormal nervous system physiology [HP:0012638]          185/244  75.819672   
Abnormality of the nervous system [HP:0000707]           185/244  75.819672   
Intellectual disability [HP:0001249]                     144/238  60.504202   
Phenotypic abnormality [HP:0000118]                      217/251  86.454183   
All [HP:0000001]                                         217/251  86.454183   
Abnormality of mental function [HP:0011446]              160/240  66.666667   
Global developmental delay [HP:0001263]                  119/204  58.333333   
Neurodevelopmental delay [HP:0012758]                    119/204  58.333333   
Abnormality of the ear [HP:0000598]                      101/209  48.325359   
Abnormal hair morphology [HP:0001595]                     95/210  45.238095   
Abnormal ocular adnexa morphology [HP:0030669]            95/210  45.238095   
Abnormal skin adnexa morphology [HP:0011138]              95/210  45.238095   
Abnormality of the ocular adnexa [HP:0032039]             95/210  45.238095   
Abnormality of the integument [HP:0001574]                95/210  45.238095   
Abnormality of the orbital region [HP:0000315]            95/210  45.238095   
Abnormality of the face [HP:0000271]                     193/242  79.752066   
Abnormality of the head [HP:0000234]                     193/242  79.752066   
Abnormality of head or neck [HP:0000152]                 193/242  79.752066   
Abnormal oral cavity morphology [HP:0000163]             180/240  75.000000   
Abnormal oral morphology [HP:0031816]                    180/240  75.000000   
Abnormality of the mouth [HP:0000153]                    180/240  75.000000   
Triangular face [HP:0000325]                              62/194  31.958763   
Abnormal facial shape [HP:0001999]                        62/194  31.958763   
Hearing abnormality [HP:0000364]                          67/198  33.838384   
Abnormal ear physiology [HP:0031704]                      67/198  33.838384   
Long philtrum [HP:0000343]                                92/216  42.592593   
Abnormality of the philtrum [HP:0000288]                  92/216  42.592593   
Abnormal upper lip morphology [HP:0000177]                92/216  42.592593   
Abnormal lip morphology [HP:0000159]                      92/216  42.592593   
Abnormal external nose morphology [HP:0010938]            99/212  46.698113   
Abnormal nasal morphology [HP:0005105]                    99/212  46.698113   
Abnormality of the nose [HP:0000366]                      99/212  46.698113   
Abnormal ear morphology [HP:0031703]                      52/196  26.530612   
Atypical behavior [HP:0000708]                            64/190  33.684211   
Short stature [HP:0004322]                               101/229  44.104803   
Abnormality of body height [HP:0000002]                  101/229  44.104803   
Growth delay [HP:0001510]                                101/229  44.104803   
Growth abnormality [HP:0001507]                          101/229  44.104803   
Macrodontia [HP:0001572]                                 159/236  67.372881   
Abnormal dental morphology [HP:0006482]                  159/236  67.372881   
Abnormality of the dentition [HP:0000164]                159/236  67.372881   
Abnormality of the hand [HP:0001155]                     134/241  55.601660   
Abnormality of the upper limb [HP:0002817]               134/241  55.601660   
Abnormality of limbs [HP:0040064]                        134/241  55.601660   
Cognitive impairment [HP:0100543]                         44/187  23.529412   

>=1 allele of the variant 16_89284634_89284639_GTGTTT_G     Yes             \
                                                          Count    Percent   
Neurodevelopmental abnormality [HP:0012759]        

Or between subjects with one variant vs. the other variant.

In [13]:
var_double = analysis.compare_by_variant_keys('16_89284129_89284134_CTTTTT_C', '16_89284634_89284639_GTGTTT_G')
var_double.summarize(hpo, BooleanPredicate.YES)

>=1 allele of either variant 16_89284129_89284134_CTTTTT_C or variant 16_89284634_89284639_GTGTTT_G  First  \
                                                                                                     Count   
Cognitive impairment [HP:0100543]                                                                     0/35   
Short stature [HP:0004322]                                                                            5/40   
Abnormality of body height [HP:0000002]                                                               5/40   
Growth delay [HP:0001510]                                                                             5/40   
Growth abnormality [HP:0001507]                                                                       5/40   
Hearing abnormality [HP:0000364]                                                                      3/36   
Abnormal ear physiology [HP:0031704]                                                                  3/36   
Abnormality of the ear [HP:0000598]                                                                   4/36   
Abnormality of the hand [HP:0001155]                                                                  8/43   
Abnormality of the upper limb [HP:0002817]                                                            8/43   
Abnormality of limbs [HP:0040064]                                                                     8/43   
Abnormal external nose morphology [HP:0010938]                                                        5/40   
Abnormal nasal morphology [HP:0005105]                                                                5/40   
Abnormality of the nose [HP:0000366]                                                                  5/40   
Long philtrum [HP:0000343]                                                                            4/39   
Abnormality of the philtrum [HP:0000288]                                                              4/39   
Abnormal upper lip morphology [HP:0000177]                                                            4/39   
Abnormal lip morphology [HP:0000159]                                                                  4/39   
Abnormality of mental function [HP:0011446]                                                           6/42   
Atypical behavior [HP:0000708]                                                                        2/36   
Abnormal nervous system physiology [HP:0012638]                                                       7/43   
Abnormality of the nervous system [HP:0000707]                                                        7/43   
Phenotypic abnormality [HP:0000118]                                                                  10/43   
All [HP:0000001]                                                                                     10/43   
Global developmental delay [HP:0001263]                                                               4/38   
Neurodevelopmental delay [HP:0012758]                                                                 4/38   
Neurodevelopmental abnormality [HP:0012759]                                                           6/43   
Triangular face [HP:0000325]                                                                          2/36   
Abnormal facial shape [HP:0001999]                                                                    2/36   
Abnormality of the face [HP:0000271]                                                                  9/43   
Abnormality of the head [HP:0000234]                                                                  9/43   
Abnormality of head or neck [HP:0000152]                                                              9/43   
Macrodontia [HP:0001572]                                                                              8/41   
Abnormal dental morphology [HP:0006482]                                                               8/41   
Abnormality of the dentition [HP:0000164

TODO - finalize!